In [5]:
import pandas as pd
import sklearn
import keras
import numpy as np

In [20]:
# df = pd.read_csv('new_PM2.5_dataset.csv') # preprocessed and sorted
df = pd.read_csv('PM2.5_dataset.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.head()

,station_id,stime,air_data_value,RH,UGRD,VGRD,HPBL,TMP,goes_measurement
0,06-011-0007,2019-01-02 20:00:00,17.0,31.600000,-2.106623,-1.797583,256.61905,282.8188,-0.005922
1,06-019-0500,2019-01-02 20:00:00,13.0,62.200000,1.205877,1.764917,337.49405,281.6313,0.087090
2,06-061-0003,2019-01-02 20:00:00,21.0,61.500000,1.518377,1.014917,270.61905,280.1313,0.094333
3,06-073-1201,2019-01-02 20:00:00,6.0,15.400001,2.080877,-1.610083,1009.30660,288.1938,-0.024185
4,06-079-2004,2019-01-02 20:00:00,7.0,50.700000,2.393377,-1.172583,460.43155,285.1938,-0.014013


### Sort by station_id and stime

In [21]:
df.sort_values(by =['station_id', 'stime'])

,station_id,stime,air_data_value,RH,UGRD,VGRD,HPBL,TMP,goes_measurement
0,06-011-0007,2019-01-02 20:00:00,17.0,31.6,-2.106623,-1.797583,256.619050,282.81880,-0.005922
11,06-011-0007,2019-01-02 21:00:00,13.0,29.7,-1.850204,-1.558695,261.172200,283.30810,0.007718
21,06-011-0007,2019-01-02 22:00:00,12.0,30.6,-1.785869,-0.232563,267.256680,283.81530,-0.036437
30,06-011-0007,2019-01-02 23:00:00,9.0,28.4,-1.616623,0.305026,216.315490,284.47230,-0.050000
81,06-011-0007,2019-01-03 19:00:00,21.0,59.8,-1.234421,0.071434,124.361984,279.52660,0.039313
...,...,...,...,...,...,...,...,...,...
30418,56-039-1013,2019-08-30 17:00:00,3.0,55.0,2.466133,1.142420,665.376300,287.81610,0.185881
30502,56-039-1013,2019-08-31 16:00:00,5.4,66.0,-0.575478,1.027882,157.645100,285.39597,0.060812
30508,56-039-1013,2019-08-31 17:00:00,4.1,53.8,0.344164,1.428810,567.691600,287.80325,0.226491
30560,56-039-1013,2019-08-31 22:00:00,4.1,24.1,4.530006,2.375885,1744.444100,294.63544,0.016734


### Change the column datatype to a suitable datatype 

In [22]:
df['stime']= pd.to_datetime(df['stime'])
df['station_id']= df["station_id"].astype(str)
df['air_data_value']= df["air_data_value"].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31570 entries, 0 to 31569
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   station_id        31570 non-null  object        
 1   stime             31570 non-null  datetime64[ns]
 2   air_data_value    31570 non-null  int64         
 3   RH                31570 non-null  float64       
 4   UGRD              31570 non-null  float64       
 5   VGRD              31570 non-null  float64       
 6   HPBL              31570 non-null  float64       
 7   TMP               31570 non-null  float64       
 8   goes_measurement  31570 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 2.2+ MB


## DATA PREPROCESSING 
#### rescale the EPA air data PM2.5 readings to 1-5 :
###### Good/Moderate/Unhealthy for Sensitive Groups/Unhealthy/Very Unhealthy/Hazardous)

In [23]:
new_df = df.drop(df.columns[[0, 1]], axis = 1)
new_df.head()

new_df['air_data_value'] = np.where((new_df['air_data_value'] >= 0) & (new_df['air_data_value'] <= 50), 0, new_df['air_data_value'])
new_df['air_data_value'] = np.where((new_df['air_data_value'] >= 51) & (new_df['air_data_value'] <= 100), 1, new_df['air_data_value'])
new_df['air_data_value'] = np.where((new_df['air_data_value'] >= 101) & (new_df['air_data_value'] <= 150), 2, new_df['air_data_value'])
new_df['air_data_value'] = np.where((new_df['air_data_value'] >= 151) & (new_df['air_data_value'] <= 250), 3, new_df['air_data_value'])
new_df['air_data_value'] = np.where((new_df['air_data_value'] >= 201) & (new_df['air_data_value'] <= 300), 4, new_df['air_data_value'])
new_df['air_data_value'] = np.where(new_df['air_data_value'] >= 301, 5, new_df['air_data_value'])

In [24]:
X = new_df.iloc[:, 1:].round(3)
Y = new_df.iloc[:, 0].astype(int)

num_X = X.to_numpy()
num_Y = Y.to_numpy()

X.head()

,RH,UGRD,VGRD,HPBL,TMP,goes_measurement
0,31.6,-2.107,-1.798,256.619,282.819,-0.006
1,62.2,1.206,1.765,337.494,281.631,0.087
2,61.5,1.518,1.015,270.619,280.131,0.094
3,15.4,2.081,-1.610,1009.307,288.194,-0.024
4,50.7,2.393,-1.173,460.432,285.194,-0.014


In [25]:
Y.head()

0    0
1    0
2    0
3    0
4    0
Name: air_data_value, dtype: int32

### PREPARE TRAINING AND TESTING DATASET

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(num_X, num_Y, random_state=42, test_size=0.25)

In [27]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import statsmodels.api as sm

scaler = preprocessing.RobustScaler() # Features Scaling is required for distance-based algorithms
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

### ML MODEL SETUP

In [104]:
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [107]:
# Random Forests
regr_rf = RandomForestRegressor(max_depth=20, random_state=42,
                             n_estimators=250)
# regr_rf = RandomForestRegressor()
regr_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, n_estimators=250, random_state=42)

### TRAINING

In [108]:
train_score = regr_rf.score(X_train, y_train)
test_score = regr_rf.score(X_test, y_test)
"training score: {}, testing score: {}".format(train_score.round(4)*100, test_score.round(4)*100)

'training score: 91.82000000000001, testing score: 52.059999999999995'

### SAVE THE BEST MODEL

In [17]:
import pickle

# # Save the Modle to file in the current working directory
# Pkl_Filename = "PM2.5_model_updated.pkl"  
# with open(Pkl_Filename, 'wb') as file:  
#     pickle.dump(regr_rf, file)

### LOAD THE PRETRAINED MODEL AND TEST

In [18]:
# Load the Model back from file
with open("PM2.5_model_updated.pkl", 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)
Pickled_LR_Model

RandomForestRegressor(max_depth=20, n_estimators=250, random_state=42)

In [28]:
train_score = Pickled_LR_Model.score(X_train, y_train)
test_score = Pickled_LR_Model.score(X_test, y_test)
"training score: {}, testing score: {}".format(train_score.round(4)*100, test_score.round(4)*100)

'training score: 91.82000000000001, testing score: 52.059999999999995'

In [29]:
y_true = num_Y
y_pred = Pickled_LR_Model.predict(num_X)

### INSERT NEW DATA FROM AQS WEBSITE

In [224]:
import requests as re
# res = re.get('https://aqs.epa.gov/data/api/monitors/bySite?email=test@aqs.api&key=test&param=88101&bdate=20190101&edate=20191231&state={}&county={}&site={}'.format(stations_ids[0][0], stations_ids[0][1], stations_ids[0][2]))
# data = res.json()['Data'][0]

# res = re.get('https://aqs.epa.gov/data/api/monitors/bySite?email=test@aqs.api&key=test&param=88101&bdate=20190101&edate=20191231&state={}&county={}&site={}'.format(stations_ids[0][0], stations_ids[0][1], stations_ids[0][2]))
station_ids = df['station_id']
stations_ids = [sid.split('-') for sid in station_ids]

def form(i, data):
    return [station_ids[i], data['latitude'], data['longitude'], data['local_site_name'], data['address'], data['state_name'], data['county_name'], data['city_name']]

db = []
print(len(stations_ids))
for i, sid in enumerate(stations_ids):
    print(i)
    res = re.get('https://aqs.epa.gov/data/api/monitors/bySite?email=chengchris8715@gmail.com&key=greyfrog63&param=88101&bdate=20190101&edate=20191231&state={}&county={}&site={}'.format(stations_ids[i][0], stations_ids[i][1], stations_ids[i][2]))
    data = res.json()['Data'][0]
    db.append(form(i, data))
    
len(db)

In [283]:
ndf = pd.DataFrame(data=db, columns=['station_id', 'latitude', 'longitude', 'local_site_name', 'address', 'state_name', 'county_name', 'city_name'])
new_new_df = pd.merge(df, ndf)
new_new_df.to_csv('new_PM2.5_dataset.csv', index=None)
new_new_df.columns

In [34]:
nndf = pd.read_csv('new_PM2.5_dataset.csv')
nndf.head()

nndf['predicted_air_pollution_level'] = y_pred.astype(int)

In [36]:
nndf.head()
nndf.to_csv('new_new_PM2.5_dataset.csv', index=None)

In [40]:
nndf.state_name.unique()

array(['California', 'Nebraska', 'Nevada', 'Oregon', 'Texas', 'Missouri',
       'New Mexico', 'Rhode Island', 'Pennsylvania', 'Oklahoma', 'Kansas',
       'South Carolina', 'District Of Columbia', 'Utah', 'Idaho',
       'Wyoming', 'Colorado', 'Montana'], dtype=object)